# Imports and types

In [1]:
import random
import time
import numpy as np
import pandas as pd
import psutil
import os 
import glob

In [2]:
def calcular_aptidao(individuo):
    return sum(individuo)

In [3]:
def mutacao(solucao):
    nova = solucao[:]
    a, b = random.sample(range(len(nova)), 2)
    nova[a], nova[b] = nova[b], nova[a]
    return nova

In [4]:
# gerar_vizinhanca
def gerar_vizinho(solucao):
    return mutacao(solucao)

In [5]:
def gerar_vizinhanca_1(solucao):
    vizinhanca = []
    for i in range(len(solucao) - 1):
        vizinho = solucao[:]
        vizinho[i], vizinho[i+1] = vizinho[i+1], vizinho[i]
        vizinhanca.append(vizinho)
    return vizinhanca

gerar_vizinhanca_1([1, 2, 3, 4])

[[2, 1, 3, 4], [1, 3, 2, 4], [1, 2, 4, 3]]

In [6]:
def gerar_vizinhanca_3(solucao):
    vizinhanca = []
    n = len(solucao)
    for i in range(n):
        for j in range(i + 1, n):
            vizinho = solucao[:]
            vizinho[i], vizinho[j] = vizinho[j], vizinho[i]
            vizinhanca.append(vizinho)
    return vizinhanca

gerar_vizinhanca_3([1, 2, 3, 4])

[[2, 1, 3, 4],
 [3, 2, 1, 4],
 [4, 2, 3, 1],
 [1, 3, 2, 4],
 [1, 4, 3, 2],
 [1, 2, 4, 3]]

In [7]:
def gerar_vizinhanca_2(solucao):
    vizinhanca = []
    for i in range(len(solucao) - 2):
        vizinho = solucao[:]
        vizinho[i], vizinho[i+2] = vizinho[i+2], vizinho[i]
        vizinhanca.append(vizinho)
    return vizinhanca

gerar_vizinhanca_2([1, 2, 3, 4])

[[3, 2, 1, 4], [1, 4, 3, 2]]

In [8]:
def gerar_array_replicavel(seed: int, tamanho: int) -> list[int]:
    random.seed(seed)
    vetor = list(range(tamanho))  
    random.shuffle(vetor)
    return vetor

In [9]:
def calcular_aptidao_qap(solucao, matriz_fluxo, matriz_distancia):
    n = len(solucao)
    custo = 0
    for i in range(n):
        for j in range(n):
            custo += matriz_fluxo[i][j] * matriz_distancia[solucao[i]][solucao[j]]
    return custo

In [10]:
def calcular_melhor_vizinho(vizinhanca, matriz_fluxo, matriz_distancia):
    melhor_vizinho = vizinhanca[0]
    for i in range(1, len(vizinhanca)):
        if calcular_aptidao_qap(vizinhanca[i], matriz_fluxo, matriz_distancia) < calcular_aptidao_qap(melhor_vizinho, matriz_fluxo, matriz_distancia):
            melhor_vizinho = vizinhanca[i]
    return melhor_vizinho

In [11]:
def VND(solucao, matriz_fluxo, matriz_distancia):
    solucao_atual = solucao[:]
    k = 0
    funcoes_de_vizinhanca = [gerar_vizinhanca_1, gerar_vizinhanca_2, gerar_vizinhanca_3]
    kmax = len(funcoes_de_vizinhanca)
    while k < kmax:
        vizinhanca = funcoes_de_vizinhanca[k](solucao_atual)
        melhor_vizinho = calcular_melhor_vizinho(vizinhanca, matriz_fluxo, matriz_distancia)
        if calcular_aptidao_qap(melhor_vizinho, matriz_fluxo, matriz_distancia) < calcular_aptidao_qap(solucao_atual, matriz_fluxo, matriz_distancia):
            solucao_atual = melhor_vizinho
            k = 0
        else:
            k += 1
    return solucao_atual

In [12]:
def inicializar_populacao(lambd, solucao_inicial):
    return [mutacao(solucao_inicial) for _ in range(lambd)]

In [13]:
def ES_VND(mu, lambd, tempo_max, taxa_mutacao, taxa_busca_local, iter_sem_melhora_max,
           solucao_inicial, matriz_fluxo, matriz_distancia):

    # Inicializa população
    P = inicializar_populacao(lambd, solucao_inicial)
    melhor = None
    melhor_aptidao = float('inf')
    sem_melhora = 0

    inicio = time.time()

    while (time.time() - inicio) < tempo_max and sem_melhora < iter_sem_melhora_max:
        aptidoes = [calcular_aptidao_qap(ind, matriz_fluxo, matriz_distancia) for ind in P]

        for ind, apt in zip(P, aptidoes):
            if apt < melhor_aptidao:
                melhor_aptidao = apt
                melhor = ind[:]
                sem_melhora = 0  
        sem_melhora += 1 

        melhores_indices = sorted(range(len(P)), key=lambda i: aptidoes[i])[:mu]
        Q = [P[i][:] for i in melhores_indices]

        nova_geracao = Q[:]
        for q in Q:
            for _ in range(lambd // mu):
                individuo = q[:]
                if random.random() < taxa_mutacao:
                    individuo = mutacao(individuo) 
                if random.random() < taxa_busca_local:
                    individuo = VND(individuo, matriz_fluxo, matriz_distancia) 
                nova_geracao.append(individuo)

        P = nova_geracao  

    return melhor, melhor_aptidao

In [14]:
def ler_qap_com_n(caminho: str):
    with open(caminho, "r") as f:
        dados = list(map(int, f.read().split()))
        
    n = dados[0]
    valores = dados[1:]  

    total_esperado = 2 * n * n
    if len(valores) != total_esperado:
        raise ValueError(f"Esperado {total_esperado} valores, mas encontrado {len(valores)}.")

    flow_flat = valores[:n * n]
    dist_flat = valores[n * n:]

    flow_df = pd.DataFrame([flow_flat[i * n:(i + 1) * n] for i in range(n)])
    dist_df = pd.DataFrame([dist_flat[i * n:(i + 1) * n] for i in range(n)])

    return n, flow_df, dist_df

In [ ]:
if __name__ == "__main__":
    arquivos = glob.glob("*.txt")

    for arquivo in arquivos:
        nome_instancia = os.path.splitext(os.path.basename(arquivo))[0]

        n, flow_df, dist_df = ler_qap_com_n(arquivo)
        flow = flow_df.values.tolist()
        dist = dist_df.values.tolist()
        matriz_fluxo = np.array(flow)
        matriz_distancia = np.array(dist)

        resultados = []  

        for seed in range(42, 52):
            random.seed(seed)

            process = psutil.Process(os.getpid())
            tempo_inicio = time.time()

            solucao_inicial = gerar_array_replicavel(seed=seed, tamanho=n)

            parametros = {
                "mu": random.randint(5, 10),
                "lambd": random.randint(30, 100),
                "tempo_max": random.randint(3, 10) * 60,
                "taxa_mutacao": random.uniform(0.4, 0.7),
                "taxa_busca_local": random.uniform(0.4, 0.7),
                "iter_sem_melhora_max": random.randint(5, 10),
                "solucao_inicial": solucao_inicial,
                "matriz_fluxo": matriz_fluxo,
                "matriz_distancia": matriz_distancia
            }

            melhor_solucao, melhor_valor = ES_VND(**parametros)

            tempo_fim = time.time()

            try:
                memoria_usada = process.memory_info().peak_wset / (1024 * 1024)  
            except AttributeError:
                memoria_usada = process.memory_info().rss / (1024 * 1024)  

            resultados.append({
                "instancia": nome_instancia,
                "seed": seed,
                "melhor_solucao": melhor_solucao,
                "custo": melhor_valor,
                "tempo_execucao_segundos": round(tempo_fim - tempo_inicio, 2),
                "memoria_usada_MB": round(memoria_usada, 2)
            })

            print(f"[{nome_instancia}] Seed {seed} finalizada. Custo: {melhor_valor} | Memória usada: {round(memoria_usada, 2)} MB")

        df_resultados = pd.DataFrame(resultados)
        df_resultados.to_csv(f"resultados_{nome_instancia}.csv", index=False)

        print(f"\nResultados da instância {nome_instancia} salvos em resultados_{nome_instancia}.csv")


[Chr12a] Seed 42 finalizada. Custo: 10476
[Chr12a] Seed 43 finalizada. Custo: 9552
[Chr12a] Seed 44 finalizada. Custo: 10096
[Chr12a] Seed 45 finalizada. Custo: 10096
[Chr12a] Seed 46 finalizada. Custo: 11248
[Chr12a] Seed 47 finalizada. Custo: 10192
[Chr12a] Seed 48 finalizada. Custo: 10192
[Chr12a] Seed 49 finalizada. Custo: 9552
[Chr12a] Seed 50 finalizada. Custo: 9552
[Chr12a] Seed 51 finalizada. Custo: 9552

Resultados da instância Chr12a salvos em resultados_Chr12a.csv
[Chr12b] Seed 42 finalizada. Custo: 10102
[Chr12b] Seed 43 finalizada. Custo: 10102
[Chr12b] Seed 44 finalizada. Custo: 9742
[Chr12b] Seed 45 finalizada. Custo: 9742
[Chr12b] Seed 46 finalizada. Custo: 9742
[Chr12b] Seed 47 finalizada. Custo: 10102
[Chr12b] Seed 48 finalizada. Custo: 10102
[Chr12b] Seed 49 finalizada. Custo: 10102
[Chr12b] Seed 50 finalizada. Custo: 9742
[Chr12b] Seed 51 finalizada. Custo: 10102

Resultados da instância Chr12b salvos em resultados_Chr12b.csv
